### SMOTE

In [15]:
# ACC 측정
y_pred = []

# Test 기간 설정 : 2019년 2월 1일 ~ 2020년 1월 31일
import pandas as pd
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
test_days = list(data['2019-01-01':'2019-12-31'].index.values)

for test_day in test_days :
    
    # 데이터 불러오기
    import pandas as pd
    data = pd.read_csv('../data/er_data.csv')

    # Dataset Columns
    data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']

    # Dataset Index
    data = data.set_index('DATE')
    
    # 데이터 shift 및 라벨링
    data['USD_1'] = data['USD'].shift(-1)
    data['Target'] = data['USD_1'] - data['USD'] * 1.001
    data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
    data = data.dropna()
    
    # Feature Selecting
    data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
    
    # 데이터 갱신
    data = data[:test_day]
    
    # Test 데이터 기준 최근 200일만 학습에 사용. 하루 예측하고, 모델 갱신
    data = data[-201:]
    train = data[:-1]
    test = data[-1:]
    
    # Target, Input Split
    train_input = train[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    train_target = train['Target']

    test_input = test[['CAD',"CHF","EUR",'GBP','JPY(100)','BITCOIN','TETHER']]
    test_target = test['Target']

    # Transform to Numpy Array
    import numpy as np
    train = np.array(train)
    test = np.array(test)
    
    # SMOTE
    from imblearn.over_sampling import SMOTE

    # 모델설정
    sm = SMOTE()

    # train데이터를 넣어 복제함
    X_resampled, y_resampled = sm.fit_sample(train_input, list(train_target))

    print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
    print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

    print("After OverSampling, counts of label '1': {}".format(sum(y_resampled=='UP')))
    print("After OverSampling, counts of label '0': {}".format(sum(y_resampled=='DOWN')))
    
    
        # Data Normalization
    mean = train_input.mean(axis=0)
    train_input -= mean
    std = train_input.std(axis=0)
    train_input /= std
    test_input -= mean
    test_input /= std
    
    # XGBoost Build
    from xgboost import XGBClassifier
    model = XGBClassifier(booster='gbtree', 
                          colsample_bylevel=0.9, 
                          colsample_bytree=0.8, 
                          gamma=0, 
                          max_depth=5, 
                          min_child_weight=3, 
                          n_estimators=10, 
                          nthread=-1, 
                          objective='binary:logistic', 
                          random_state=0, 
                          silent= True,
                          learning_rate = 0.1)

    # Model Fitting
    model.fit(train_input,train_target, eval_set=[(train_input, train_target)], early_stopping_rounds=50,verbose=0)
    
    # Acc Check
    from sklearn.metrics import accuracy_score
    y_pred.append(model.predict(test_input)[0])
    
# y_true
data = pd.read_csv('../data/er_data.csv')
data.columns = ['DATE','CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','USD']
data = data.set_index('DATE')
data['USD_1'] = data['USD'].shift(-1)
data['Target'] = data['USD_1'] - data['USD'] * 1.001
data['Target'] = data['Target'].apply(lambda x : 'UP' if x > 0 else 'DOWN')
data = data.dropna()
data = data[['CAD','CHF','EUR','GBP','JPY(100)','BITCOIN','TETHER','Target']]
y_true = data['2019-01-02':'2019-12-31']['Target']

# ACC
print('Online Learning ACC = {:.2f}%'.format(accuracy_score(y_true,y_pred)*100))

After OverSampling, the shape of train_X: (242, 7)
After OverSampling, the shape of train_y: (242, 7) 

After OverSampling, counts of label '1': 121
After OverSampling, counts of label '0': 121
After OverSampling, the shape of train_X: (242, 7)
After OverSampling, the shape of train_y: (242, 7) 

After OverSampling, counts of label '1': 121
After OverSampling, counts of label '0': 121
After OverSampling, the shape of train_X: (240, 7)
After OverSampling, the shape of train_y: (240, 7) 

After OverSampling, counts of label '1': 120
After OverSampling, counts of label '0': 120
After OverSampling, the shape of train_X: (242, 7)
After OverSampling, the shape of train_y: (242, 7) 

After OverSampling, counts of label '1': 121
After OverSampling, counts of label '0': 121
After OverSampling, the shape of train_X: (242, 7)
After OverSampling, the shape of train_y: (242, 7) 

After OverSampling, counts of label '1': 121
After OverSampling, counts of label '0': 121
After OverSampling, the shape 